In [1]:
import pandas as pd

In [2]:
census = pd.read_csv("census_data.csv")

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [6]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_data = census.drop('income_bracket',axis=1)
y_labels = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

In [9]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [10]:
import tensorflow as tf

In [11]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [12]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [13]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

In [14]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

**If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info**

In [15]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_eval_distribute': None, '_train_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa36978c898>, '_model_dir': '/tmp/tmpq8zbu83f', '_num_ps_replicas': 0, '_evaluation_master': '', '_service': None, '_protocol': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_task_type': 'worker', '_tf_random_seed': None, '_master': '', '_experimental_distribute': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_is_chief': True, '_save_checkpoints_secs': 600, '_device_fn': None, '_task_id': 0}


In [16]:
model.train(input_fn=input_func,steps=5000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpq8zbu83f/model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 117.052
INFO:tensorflow:loss = 871.13635, step = 101 (0.854 sec)
INFO:tensorflow:global_step/sec: 196.433
INFO:tensorflow:loss = 136.3235, step = 201 (0.509 sec)
INFO:tensorflow:global_step/sec: 197.02
INFO:tensorflow:loss = 150.08852, step = 301 (0.509 sec)
INFO:tensorflow:global_step/sec: 178.513
INFO:tensorflow:loss = 410.18982, step = 401 (0.562 sec)
INFO:te

In [17]:

pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [18]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpq8zbu83f/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [19]:
predictions[0]

{'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'logistic': array([0.21997572], dtype=float32),
 'logits': array([-1.2658079], dtype=float32),
 'probabilities': array([0.7800243 , 0.21997574], dtype=float32)}

In [20]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [21]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [22]:
from sklearn.metrics import classification_report

In [23]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89      7436
           1       0.65      0.61      0.63      2333

   micro avg       0.83      0.83      0.83      9769
   macro avg       0.76      0.75      0.76      9769
weighted avg       0.82      0.83      0.83      9769

